<a href="https://colab.research.google.com/github/MaiKhoa0101/MachineLearning/blob/main/PhoBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j -q
!pip install Cython
!pip install -U fasttext --no-cache-dir --no-deps --force-reinstall
# ==================== CÀI ĐẶT ====================
# Chạy cell này trước
!pip install -q flask flask-cors pyngrok transformers torch


print("✅ Đã cài đặt xong các thư viện")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from neo4j import GraphDatabase
import numpy as np
from typing import List, Dict, Tuple
import warnings
from flask import Flask, request, jsonify
from flask_cors import CORS

warnings.filterwarnings('ignore')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

🖥️  Device: cpu


In [ ]:
print("📥 Đang tải PhoBERT-large...")

# Load tokenizer và model
MODEL_NAME = "vinai/phobert-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
model = model.to(device)
model.eval()

print("✅ Đã load PhoBERT-large thành công!")
print(f"   Vocab size: {tokenizer.vocab_size:,}")
print(f"   Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
import os
import time
import torch
import warnings
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForMaskedLM

# ==================== 1. RESET NGROK ====================
print("🛑 Đang reset Ngrok...")

try:
    os.system("killall ngrok 2>/dev/null")  # Tắt process local
    ngrok.kill()  # Tắt qua thư viện
    time.sleep(3)  # Đợi cleanup
    print("✅ Đã reset Ngrok local")
except:
    print("⚠️  Không có Ngrok process nào đang chạy")

# ==================== 2. LOAD MODEL ====================
print("\n⏳ Đang tải PhoBERT model...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

try:
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
    model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-large")
    model = model.to(device)
    model.eval()
    print(f"✅ Đã tải xong PhoBERT! (Device: {device})\n")
except Exception as e:
    print(f"❌ Lỗi tải model: {e}")
    exit(1)

# ==================== 3. FLASK APP ====================
app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'ok',
        'model': 'PhoBERT-large',
        'device': str(device),
        'vocab_size': tokenizer.vocab_size
    })

@app.route('/predict', methods=['POST'])
def predict():
    """
    Word prediction endpoint

    Request body:
    {
        "context": "Tôi đang",
        "candidates": ["học", "làm", "ăn", "ngủ"],
        "top_k": 5
    }
    """
    try:
        data = request.json
        context = data.get('context', '').strip()
        candidates = data.get('candidates', [])
        top_k = data.get('top_k', 10)

        # Validation
        if not context:
            return jsonify({
                'success': False,
                'message': 'Context không được rỗng'
            }), 400

        if not candidates:
            return jsonify({
                'success': False,
                'message': 'Candidates không được rỗng'
            }), 400

        # Thêm mask token
        masked_text = f"{context} {tokenizer.mask_token}"

        # Tokenize
        inputs = tokenizer(masked_text, return_tensors="pt").to(device)

        # Tìm vị trí mask
        mask_indices = torch.where(
            inputs["input_ids"] == tokenizer.mask_token_id
        )[1]

        if len(mask_indices) == 0:
            return jsonify({
                'success': False,
                'message': 'Không tìm thấy vị trí mask token'
            }), 400

        mask_token_index = mask_indices[0]

        # Dự đoán
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Tính probabilities
        mask_logits = logits[0, mask_token_index, :]
        probs = torch.softmax(mask_logits, dim=-1)

        # Score từng candidate
        predictions = []
        for candidate in candidates:
            # Tokenize candidate (lấy token đầu tiên)
            tokens = tokenizer.encode(
                candidate,
                add_special_tokens=False
            )

            if len(tokens) > 0:
                token_id = tokens[0]
                score = probs[token_id].item()
                predictions.append({
                    "word": candidate,
                    "score": float(score),
                    "token_id": int(token_id)
                })

        # Sort theo score giảm dần
        predictions.sort(key=lambda x: x['score'], reverse=True)

        return jsonify({
            'success': True,
            'context': context,
            'masked_text': masked_text,
            'predictions': predictions[:top_k],
            'total_candidates': len(candidates)
        })

    except Exception as e:
        print(f"❌ Lỗi xử lý request: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({
            'success': False,
            'message': str(e)
        }), 500

# ==================== 4. RUN SERVER ====================
if __name__ == '__main__':
    NGROK_TOKEN = "37ji3NBafQsRSv0DACn5Vkf8mFD_HramkUED3pcgezorryEF"
    ngrok.set_auth_token(NGROK_TOKEN)

    PORT = 5010

    try:
        print("🔌 Đang kết nối Ngrok...")

        # FIX: Không chỉ định domain để tránh conflict
        # Ngrok sẽ tự động tạo domain ngẫu nhiên mỗi lần
        public_url_obj = ngrok.connect(
            PORT,
            bind_tls=True  # Chỉ dùng HTTPS
        )

        public_url = public_url_obj.public_url

        print("\n" + "="*70)
        print("✅ PHOBERT SERVER ĐANG CHẠY THÀNH CÔNG!")
        print("="*70)
        print(f"📡 Public URL:  {public_url}")
        print(f"🏥 Health Check: {public_url}/health")
        print(f"🎯 Predict API:  {public_url}/predict")
        print("="*70)
        print("\n💡 Tip: URL này sẽ thay đổi mỗi lần khởi động lại")
        print("    Nếu muốn domain cố định, nâng cấp tài khoản Ngrok\n")

        # Chạy Flask
        app.run(host='0.0.0.0', port=PORT, debug=False)

    except Exception as e:
        print("\n" + "="*70)
        print("❌ LỖI KHỞI ĐỘNG NGROK")
        print("="*70)
        print(f"Chi tiết: {str(e)}\n")

        if "already online" in str(e):
            print("🔧 CÁCH FIX:")
            print("   1. Vào: https://dashboard.ngrok.com/agents/sessions")
            print("   2. Tìm session đang chạy và bấm 'Stop'")
            print("   3. Chạy lại script này")
            print("\n   Hoặc đợi 5-10 phút để session tự hết hạn\n")

        print("="*70)

🛑 Đang reset Ngrok...
✅ Đã reset Ngrok local

⏳ Đang tải PhoBERT model...


Some weights of the model checkpoint at vinai/phobert-large were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ Đã tải xong PhoBERT! (Device: cpu)

🔌 Đang kết nối Ngrok...

✅ PHOBERT SERVER ĐANG CHẠY THÀNH CÔNG!
📡 Public URL:  https://veinless-unslanderously-jordyn.ngrok-free.dev
🏥 Health Check: https://veinless-unslanderously-jordyn.ngrok-free.dev/health
🎯 Predict API:  https://veinless-unslanderously-jordyn.ngrok-free.dev/predict

💡 Tip: URL này sẽ thay đổi mỗi lần khởi động lại
    Nếu muốn domain cố định, nâng cấp tài khoản Ngrok

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5010
 * Running on http://172.28.0.12:5010
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Tắt tất cả các tunnel đang chạy
ngrok.kill()
class PhoBERTNextWordPredictor:
    """Predictor sử dụng PhoBERT để dự đoán từ tiếp theo"""

    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.mask_token = tokenizer.mask_token

    def predict_next_word(self,
                         context: str,
                         top_k: int = 10,
                         filter_words: List[str] = None) -> List[Dict]:
        """
        Dự đoán từ tiếp theo dựa trên context

        Args:
            context: Câu/cụm từ hiện tại (VD: "Tôi đang")
            top_k: Số lượng từ gợi ý
            filter_words: Danh sách từ cần filter (optional)

        Returns:
            List of {word, score, rank}
        """
        # Thêm mask token vào cuối context
        masked_text = f"{context} {self.mask_token}"

        # Tokenize
        inputs = self.tokenizer(masked_text, return_tensors="pt").to(self.device)

        # Get mask token position
        mask_token_index = torch.where(inputs["input_ids"] == self.tokenizer.mask_token_id)[1]

        # Predict
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits

        # Get predictions for mask token
        mask_token_logits = logits[0, mask_token_index, :]

        # Convert to probabilities
        probs = torch.softmax(mask_token_logits, dim=-1)

        # Get top k predictions
        top_k_probs, top_k_indices = torch.topk(probs, k=min(top_k * 3, 100), dim=-1)

        # Decode tokens
        predictions = []
        for i in range(top_k_indices.shape[1]):
            token_id = top_k_indices[0, i].item()
            token = self.tokenizer.decode([token_id]).strip()
            score = top_k_probs[0, i].item()

            # Filter out special tokens và subwords
            if token.startswith('##') or token in ['<s>', '</s>', '<pad>', '<unk>']:
                continue

            # Filter custom words if provided
            if filter_words and token.lower() in [w.lower() for w in filter_words]:
                continue

            predictions.append({
                'word': token,
                'score': score,
                'rank': len(predictions) + 1
            })

            if len(predictions) >= top_k:
                break

        return predictions

    def predict_multiple_positions(self,
                                   context: str,
                                   num_words: int = 3,
                                   top_k_per_position: int = 5) -> List[List[Dict]]:
        """
        Dự đoán nhiều từ tiếp theo (sequence prediction)

        Returns:
            List of predictions for each position
        """
        all_predictions = []
        current_context = context

        for i in range(num_words):
            predictions = self.predict_next_word(current_context, top_k=top_k_per_position)

            if not predictions:
                break

            all_predictions.append(predictions)

            # Extend context with best prediction
            best_word = predictions[0]['word']
            current_context = f"{current_context} {best_word}"

        return all_predictions


# Khởi tạo predictor
phobert_predictor = PhoBERTNextWordPredictor(model, tokenizer, device)
print("✅ PhoBERT Predictor đã sẵn sàng!")

In [ ]:
def print_predictions(predictions: List[Dict], title: str = "Predictions"):
    """In kết quả dự đoán đẹp mắt"""
    print(f"\n{'='*60}")
    print(f"📊 {title}")
    print(f"{'='*60}")

    for pred in predictions:
        rank = pred.get('rank', '?')
        word = pred['word']

        if 'combined_score' in pred:
            # Hybrid predictions
            combined = pred['combined_score']
            phobert = pred.get('phobert_score', 0)
            neo4j = pred.get('neo4j_score', 0)
            print(f"{rank:2d}. {word:15s} | Combined: {combined:.4f} | PhoBERT: {phobert:.4f} | Neo4j: {neo4j:.4f}")
        else:
            # Simple predictions
            score = pred.get('score', 0)
            print(f"{rank:2d}. {word:15s} | Score: {score:.4f}")

    print(f"{'='*60}\n")


# ========== TEST CASES ==========

test_sentences = [
    "Hôm nay tôi muốn đi chơi với",
    "Hôm nay trời",
    "Anh ấy rất",
    "Chúng tôi sẽ",
    "Việt Nam là",
]

print("\n" + "="*60)
print("🧪 TESTING PHOBERT PREDICTOR")
print("="*60)

for sentence in test_sentences:
    print(f"\n📝 Context: \"{sentence}\"")
    predictions = phobert_predictor.predict_next_word(sentence, top_k=5)
    print_predictions(predictions, title=f"PhoBERT Predictions for '{sentence}'")

In [ ]:
# ==================== TEST PHOBERT API ====================
# Chạy cell này SAU KHI server đã chạy

import requests

# ⚠️ THAY ĐỔI URL này bằng URL từ output của cell trước
PUBLIC_URL = "https://your-url-here.ngrok.io"  # ← Thay đổi URL này!

print("🧪 Testing PhoBERT API...")
print(f"📡 URL: {PUBLIC_URL}\n")

# ==================== TEST 1: HEALTH CHECK ====================
print("1️⃣ Testing /health...")
try:
    response = requests.get(f"{PUBLIC_URL}/health")
    result = response.json()
    print(f"   Status: {result['status']}")
    print(f"   Model:  {result['model']}")
    print(f"   Device: {result['device']}")
    print("   ✅ Health check OK\n")
except Exception as e:
    print(f"   ❌ Error: {e}\n")

# ==================== TEST 2: PREDICT ====================
print("2️⃣ Testing /predict...")

test_cases = [
    {
        "context": "Tôi đang",
        "candidates": ["học", "làm", "ở", "nghĩ", "đi"],
        "top_k": 5
    },
    {
        "context": "Hôm nay trời",
        "candidates": ["đẹp", "xấu", "nóng", "mưa", "lạnh"],
        "top_k": 3
    },
    {
        "context": "Tôi yêu",
        "candidates": ["bạn", "em", "mẹ", "cha", "anh"],
        "top_k": 3
    }
]

for i, test in enumerate(test_cases, 1):
    print(f"\n   Test {i}: '{test['context']}'")
    print(f"   Candidates: {test['candidates']}")

    try:
        response = requests.post(
            f"{PUBLIC_URL}/predict",
            json=test
        )

        result = response.json()

        if result['success']:
            print(f"   Results:")
            for j, pred in enumerate(result['predictions'], 1):
                print(f"      {j}. {pred['word']:10s} → {pred['score']:.4f}")
            print(f"   ✅ Test {i} passed")
        else:
            print(f"   ❌ Test {i} failed: {result.get('message')}")

    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n" + "="*60)
print("✅ Testing completed!")
print("="*60)

# ==================== USAGE EXAMPLE ====================
print("\n📝 Sử dụng trong NestJS:")
print(f"""
// .env
PHOBERT_API_URL={PUBLIC_URL}
ENABLE_PHOBERT=true
HYBRID_ALPHA=0.6

// TypeScript
const result = await nlpService.getNextWordSuggestion('Tôi đang');
console.log(result.suggestions);
""")